# First things first:

## Import all the libraries that will help to clean the database

In [1]:
!pip3 install regex

In [2]:
from functools import reduce
import numpy as np
import pandas as pd
import re

## Import the CSV file 

In [3]:
df = pd.read_csv("/home/carolina/Desktop/IRONHACK/Project files/attacks.csv", encoding="latin-1")

In [4]:
df.head(15)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,...,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,...,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,...,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,...,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,...,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


## Start to explore the data

In [5]:
# Rows x Columns

df.shape

(25723, 24)

In [6]:
# Columns name

df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
# Columns that are hidden on the df above:

df[['Age', 'Injury', 'Fatal (Y/N)', 'Time']].head()

,Age,Injury,Fatal (Y/N),Time
0,57,"No injury to occupant, outrigger canoe and pad...",N,18h00
1,11,Minor injury to left thigh,N,14h00 -15h00
2,48,Injury to left lower leg from surfboard skeg,N,07h45
3,NaN,Minor injury to lower leg,N,NaN
4,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN


From this first investigation we can highlight some observations:
- The columns "Unnamed: 22" and "Unnamed: 23" seems to don't have any information. 
- The columns "Case Number 1." and "Case Number 2." looks like duplicated from the column Case Number.
- Some columns such as "pdf", "href formula", "href", "original order" might won't be usefull for the analysis.

If these obervarions are correct we can erase these columns, but before deleting any data we need to make sure.

# First Clean action: Remove duplicates

Before start to manipulate the data, let's check if we have duplicated information.

In [8]:
df.drop_duplicates()


display(df.head())
print(df.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


(25723, 24)


There aren't duplicates on the whole database, but these doesn't mean that we don't have duplicated columns. We will check these information later. 



# Second Action: Clean null values

## Delete columns with null values

Check if the column "Unnamed: 22" and "Unnamed: 23" only have null values.

In [9]:
print(df['Unnamed: 22'].notnull().sum())

print(df[df["Unnamed: 22"].notnull()]["Unnamed: 22"])

1
1478    stopped here
Name: Unnamed: 22, dtype: object


In [10]:
df['Unnamed: 23'].notnull().sum()

print(df[df["Unnamed: 23"].notnull()]["Unnamed: 23"])

4415             Teramo
5840    change filename
Name: Unnamed: 23, dtype: object


As we can see both columns don't have valuable data, so we can delete them.

In [11]:
df = df.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

#df.head()

## Clean null rows

In order to clean the rows, first we need to transpose index and columns. 

Then we will check those "columns" that are fully empty. Their sum of isnull will be 22, because this is the old number of columns that now are index.

After identify these empty columns we will drop them from the dataFrame. 

When we get back to the clean dataFrame, where the index and columns are back to normal, we can see that acctually we have deleted the null rows.

In [12]:
null_row = df.T.isnull().sum().values
null_row_index_22 = df.T.isnull().sum()[df.T.isnull().sum() == 22].index
clean_df = df.drop([e for e in null_row_index_22])


#display(clean_df.head())
print(clean_df.shape)

(8703, 22)


# Third action: Clean some columns

Let's check those columns that are not usefull for our analysis and delete them.

## Check duplicated columns

Compare the columns "Case Number", "Case Number.1" and "Case Number.2".

1º Do an analysis apart from the dataFrame.

2º Data wrangling - Change all the values of null to 0.

3º See if they are equal("true") or have diffent values("false".

4º With the result we saw that ""Case Number.1" only differ on 20 values with "Case Number.2", which we can sat that they are particully the same. Now, when compare to "Case Number" they are 2.000 difent between them.

5º Checking the different rows(those that have "false"). We can see that actually most of then are 0, the rest it is pretty much the same. This means we can delete the duplicate columns and mantain just the first one. 

6º Drop the columns on the dataFrame, not on the apart version we created for this analysis. 

In [13]:
c_df = clean_df[['Case Number','Case Number.1', 'Case Number.2']]

c_df['Case Number'].fillna(0, inplace = True) 
c_df['Case Number.1'].fillna(0, inplace = True) 
c_df['Case Number.2'].fillna(0, inplace = True)

c_df['check'] = c_df['Case Number'] == c_df['Case Number.1']
c_df['check2'] = c_df['Case Number.1'] == c_df['Case Number.2']

#print(c_df["check"].value_counts())
#print(c_df["check2"].value_counts())

h = c_df[c_df['check'] == False][['Case Number','Case Number.1', 'Case Number.2']]
#h.head()






/usr/lib/python3/dist-packages/pandas/core/generic.py:5436: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
clean_df = clean_df.drop(columns=['Case Number.1', 'Case Number.2'])

#clean_df.head()


## Delete columns we won't need for analysis

These are columns that are too descritive or that have information that won't be usefull to create any analysis, so we can delete in order to have the database cleaner. 

In [15]:
clean_df = clean_df.drop(columns=['Location', 'Name','Injury', 'Investigator or Source', 'pdf', 'href formula', 'href', 'original order'])

clean_df.head()



,Case Number,Date,Year,Type,Country,Area,Activity,Sex,Age,Fatal (Y/N),Time,Species
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,Paddling,F,57,N,18h00,White shark
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,Standing,F,11,N,14h00 -15h00,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,Surfing,M,48,N,07h45,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,M,NaN,N,NaN,2 m shark
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,Free diving,M,NaN,N,NaN,"Tiger shark, 3m"


In [16]:
# Now we only have 12 columns and the dataframe already looks much cleaner

clean_df.shape

(8703, 12)

# Four action: Clean by string matching

Now we will edit the data, so we group by on the analysis phase.

## Type: Only Unprovoked, Provoked and Not available

In [17]:
clean_df.Type.value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [18]:
clean_df["Type"] = clean_df.Type.fillna("Not Available")


clean_df.loc[clean_df["Type"].str.startswith("B"),"Type"] = "Unprovoked"
clean_df.loc[clean_df["Type"].str.startswith("S"),"Type"] = "Unprovoked"
clean_df.loc[clean_df["Type"].str.startswith("I"),"Type"] = "Not Available"
clean_df.loc[clean_df["Type"].str.startswith("Q"),"Type"] = "Not Available"

clean_df.Type.value_counts()

Unprovoked       5175
Not Available    2954
Provoked          574
Name: Type, dtype: int64

## Activity - This will be one of the mains column for our analysis

In [19]:
clean_df.Activity.value_counts().head()

Surfing         971
Swimming        869
Fishing         431
Spearfishing    333
Bathing         162
Name: Activity, dtype: int64

Aggruping the activities on the key groups bellow, by changing the activity name:

In [20]:
clean_df["Activity"] = clean_df.Activity.fillna("Not Available")

#SURFING
clean_df.loc[clean_df["Activity"].str.contains("Surf"),"Activity"] = "Surfing"
clean_df.loc[clean_df["Activity"].str.contains("Board"),"Activity"] = "Surfing"
clean_df.loc[clean_df["Activity"].str.contains("surf"),"Activity"] = "Surfing"
clean_df.loc[clean_df["Activity"].str.contains("board"),"Activity"] = "Surfing"


#Swimming
clean_df.loc[clean_df["Activity"].str.contains("Bath"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.startswith("Wad"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.contains("Swi"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.startswith("Floa"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.contains("swi"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.contains("Jump"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.contains("Play"),"Activity"] = "Swimming"
clean_df.loc[clean_df["Activity"].str.contains("Sculling"),"Activity"] = "Swimming"


#Fishing
clean_df.loc[clean_df["Activity"].str.contains("Fish"),"Activity"] = "Fishing"
clean_df.loc[clean_df["Activity"].str.contains("fish"),"Activity"] = "Fishing"
clean_df.loc[clean_df["Activity"].str.startswith("Spear"),"Activity"] = "Fishing"
clean_df.loc[clean_df["Activity"].str.contains("Clammin"),"Activity"] = "Fishing"


#Diving
clean_df.loc[clean_df["Activity"].str.contains("Div"),"Activity"] = "Diving"
clean_df.loc[clean_df["Activity"].str.contains("div"),"Activity"] = "Diving"
clean_df.loc[clean_df["Activity"].str.contains("Snor"),"Activity"] = "Diving"


#Kayaking
clean_df.loc[clean_df["Activity"].str.contains("Canoe"),"Activity"] = "Kayaking"
clean_df.loc[clean_df["Activity"].str.contains("Row"),"Activity"] = "Kayaking"
clean_df.loc[clean_df["Activity"].str.contains("Kay"),"Activity"] = "Kayaking"

#Treading water
clean_df.loc[clean_df["Activity"].str.contains("Walk"),"Activity"] = "Treading water"
clean_df.loc[clean_df["Activity"].str.contains("Dang"),"Activity"] = "Treading water"
clean_df.loc[clean_df["Activity"].str.contains("Standing"),"Activity"] = "Treading water"

#Sailing
clean_df.loc[clean_df["Activity"].str.contains("Boat"),"Activity"] = "Sailing"
clean_df.loc[clean_df["Activity"].str.contains("Ship"),"Activity"] = "Sailing"

#Fell
clean_df.loc[clean_df["Activity"].str.contains("Sea"),"Activity"] = "Fell into the water"
clean_df.loc[clean_df["Activity"].str.contains("Fell"),"Activity"] = "Fell into the water"

#Interacting with sharks
clean_df.loc[clean_df["Activity"].str.contains("Shark"),"Activity"] = "Interacting with sharks"
clean_df.loc[clean_df["Activity"].str.contains("shark"),"Activity"] = "Interacting with sharks"

#Others
clean_df.loc[clean_df["Activity"].str.contains("Suicide"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("Murder"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("Paddleskiing"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("Lifesaving drill"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("Seine"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("SUP"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("Splashing"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.contains("Escap"),"Activity"] = "Others"
clean_df.loc[clean_df["Activity"].str.startswith("."),"Activity"] = "Not Available"
clean_df.loc[clean_df["Activity"].str.contains("Escap"),"Activity"] = "Others"


All those activity that have less than 4 value_counts() can be deleted from our analysis.

In [21]:
counts_Activity = clean_df.groupby("Activity")["Activity"].transform(len)

mask = (counts_Activity > 5)

#clean_df[mask]

In [22]:
df_clean = clean_df[mask]

#df_clean

Activity clean! This is super important because this is one of the columns that we will built our analysin.

In [23]:
df_clean.Activity.value_counts()

Not Available              2948
Surfing                    1562
Swimming                   1526
Fishing                    1154
Diving                      606
Treading water              179
Interacting with sharks     121
Kayaking                     75
Sailing                      51
Fell into the water          45
Others                       40
Name: Activity, dtype: int64

## Sex: M/F

In [24]:
df_clean["Sex "].value_counts()

M      4809
F       606
M         2
N         2
.         1
lli       1
Name: Sex , dtype: int64

In [25]:
df_clean["Sex "] = df_clean["Sex "].fillna("Not Available")

df_clean.loc[df_clean["Sex "].str.startswith("N"),"Sex "] = "M"
df_clean.loc[df_clean["Sex "].str.startswith("M"),"Sex "] = "M"
df_clean.loc[df_clean["Sex "].str.startswith("."),"Sex "] = "Not Available"
df_clean.loc[df_clean["Sex "].str.startswith("O"),"Sex "] = "Not Available"
df_clean.loc[df_clean["Sex "].str.startswith("l"),"Sex "] = "Not Available"



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
#Sex is clean now!:

df_clean["Sex "].value_counts()


M                7699
F                 606
Not Available       2
Name: Sex , dtype: int64

## Fatal: Y/N

In [27]:
df_clean["Fatal (Y/N)"].value_counts()

N          4127
Y          1215
UNKNOWN      68
 N            6
N             1
2017          1
y             1
M             1
Name: Fatal (Y/N), dtype: int64

In [28]:
df_clean["Fatal (Y/N)"] = df_clean["Fatal (Y/N)"].fillna("Not Available")

df_clean.loc[df_clean["Fatal (Y/N)"].str.startswith("U"),"Fatal (Y/N)"] = "Not Available"
df_clean.loc[df_clean["Fatal (Y/N)"].str.startswith("N"),"Fatal (Y/N)"] = "N"
df_clean.loc[df_clean["Fatal (Y/N)"].str.startswith("y"),"Fatal (Y/N)"] = "Y"
df_clean.loc[df_clean["Fatal (Y/N)"].str.startswith("M"),"Fatal (Y/N)"] = "N"
df_clean.loc[df_clean["Fatal (Y/N)"].str.endswith("N"),"Fatal (Y/N)"] = "N"
df_clean.loc[df_clean["Fatal (Y/N)"].str.startswith("2"),"Fatal (Y/N)"] = "N"


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
df_clean["Fatal (Y/N)"].value_counts()

N    7091
Y    1216
Name: Fatal (Y/N), dtype: int64

## Age: Let's arrange in Teen, Young, Adult and Old

In [30]:
df_clean["Age"] = df_clean["Age"].fillna("0")


df_clean["Age"]= df_clean["Age"].astype(str) 

df_clean.loc[df_clean["Age"].str.startswith("1"), "Age"] = "Teen (1 - 19)"
df_clean.loc[df_clean["Age"].str.startswith("2"), "Age"] = "Young (20 - 29)"
df_clean.loc[df_clean["Age"].str.startswith("3"), "Age"] = "Adult (30 - 59)"
df_clean.loc[df_clean["Age"].str.startswith("4"), "Age"] = "Adult (30 - 59)"
df_clean.loc[df_clean["Age"].str.startswith("5"), "Age"] = "Adult (30 - 59)"
df_clean.loc[df_clean["Age"].str.startswith("6"), "Age"] = "Old (> 60)"
df_clean.loc[df_clean["Age"].str.startswith("7"), "Age"] = "Old (> 60)"
df_clean.loc[df_clean["Age"].str.startswith("8"), "Age"] = "Old (> 60)"
df_clean.loc[df_clean["Age"].str.startswith("9"), "Age"] = "Old (> 60)"

df_clean.loc[df_clean["Age"].str.contains("een"), "Age"] = "Teen (1 - 19)"
df_clean.loc[df_clean["Age"].str.contains("oung"), "Age"] = "Young (20 - 29)"
df_clean.loc[df_clean["Age"].str.contains("dult"), "Age"] = "Adult (30 - 59)"


df_clean.loc[df_clean["Age"].str.contains("mid"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.contains("X"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.contains("Ca"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.contains("F"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.contains("derly"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.contains("LINE"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.contains("A.M."), "Age"] = "Not Available"

df_clean.loc[df_clean["Age"].str.contains("50"), "Age"] = "Old (> 60)"
df_clean.loc[df_clean["Age"].str.contains("19"), "Age"] = "Teen (1 - 19)"
df_clean.loc[df_clean["Age"].str.contains("28"), "Age"] = "Young (20 - 29)"
df_clean.loc[df_clean["Age"].str.contains("14"), "Age"] = "Teen (1 - 19)"
df_clean.loc[df_clean["Age"].str.contains("43"), "Age"] = "Adult (30 - 59)"
df_clean.loc[df_clean["Age"].str.contains("30"), "Age"] = "Adult (30 - 59)"

df_clean.loc[df_clean["Age"].str.startswith("0"), "Age"] = "Not Available"
df_clean.loc[df_clean["Age"].str.startswith(" "), "Age"] = "Not Available"


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [31]:
df_clean["Age"].value_counts()

Not Available      4962
Teen (1 - 19)      1082
Adult (30 - 59)    1078
Young (20 - 29)     980
Old (> 60)          204
                      1
Name: Age, dtype: int64

## Country

Another column very important for our analysis. Will be difficult to clean all, but we can focus on the countries with most shark attack.

In [32]:
print(df_clean.shape)
#df_clean["Country"].value_counts()

(8307, 12)


In [33]:
df_clean["Country"] = df_clean["Country"].fillna("Not Available")

df_clean.loc[df_clean["Country"].str.contains("USA"), "Country"] = "USA"
df_clean.loc[df_clean["Country"].str.contains("AUST"), "Country"] = "AUSTRALIA"
df_clean.loc[df_clean["Country"].str.contains("AFRICA"), "Country"] = "AFRICA"
df_clean.loc[df_clean["Country"].str.contains("GUINEA"), "Country"] = "NEW GUINEA"
df_clean.loc[df_clean["Country"].str.contains("PHILIP"), "Country"] = "PHILIPPINES"
df_clean.loc[df_clean["Country"].str.contains("MEX"), "Country"] = "MEXICO"
df_clean.loc[df_clean["Country"].str.contains("ITALY"), "Country"] = "ITALY"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
print(df_clean.shape)
df_clean["Country"].value_counts().head(10)

(8307, 12)


Not Available    2444
USA              2144
AUSTRALIA        1265
AFRICA            559
NEW GUINEA        136
NEW ZEALAND       123
BRAZIL            104
BAHAMAS           102
MEXICO             85
ITALY              69
Name: Country, dtype: int64

All those Countries that have less than 40 shark attack can be deleted from our analysis.

In [35]:
counts_COUNTRY = df_clean.groupby("Country")["Country"].transform(len)

Mask = (counts_COUNTRY > 40)

#df_clean[Mask]

In [36]:
DF_Clean = df_clean[Mask]

DF_Clean["Country"].value_counts()

Not Available    2444
USA              2144
AUSTRALIA        1265
AFRICA            559
NEW GUINEA        136
NEW ZEALAND       123
BRAZIL            104
BAHAMAS           102
MEXICO             85
ITALY              69
FIJI               58
REUNION            58
NEW CALEDONIA      51
SPAIN              44
PHILIPPINES        43
MOZAMBIQUE         42
Name: Country, dtype: int64

## Species

Again, will be difficult to clean all these data because is too descritive, but we can clean up the top ones.

In [37]:
DF_Clean['Species '].value_counts().head()

White shark                                           141
Invalid                                                77
Shark involvement prior to death was not confirmed     77
Shark involvement not confirmed                        76
Tiger shark                                            56
Name: Species , dtype: int64

In [38]:
DF_Clean['Species '] = DF_Clean['Species '].fillna("Not Available")

DF_Clean.loc[DF_Clean['Species '].str.contains("hite"), 'Species '] = "White Shark"
DF_Clean.loc[DF_Clean['Species '].str.contains("iger"), 'Species '] = "Tiger Shark"
DF_Clean.loc[DF_Clean['Species '].str.contains("ull"), 'Species '] = "Bull Shark"
DF_Clean.loc[DF_Clean['Species '].str.contains("obbe"), 'Species '] = "Wobbegong Shark"
DF_Clean.loc[DF_Clean['Species '].str.contains("nvalid"), 'Species '] = "Not Available"
DF_Clean.loc[DF_Clean['Species '].str.contains("uestiona"), 'Species '] = "Not Available"
DF_Clean.loc[DF_Clean['Species '].str.contains("confirme"), 'Species '] = "Not Available"
DF_Clean.loc[DF_Clean['Species '].str.contains("nvolvement"), 'Species '] = "Not Available"
DF_Clean.loc[DF_Clean['Species '].str.contains("dentif"), 'Species '] = "Not Available"
DF_Clean.loc[DF_Clean['Species '].str.contains("shark"), 'Species '] = "Not Available"
DF_Clean.loc[DF_Clean['Species '].str.contains("Blue"), 'Species '] = "Blue Shark"
DF_Clean.loc[DF_Clean['Species '].str.startswith(" "), 'Species '] = "Not Available"


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Now, our database already looks much cleaner, but we still can shape it more.

# Fifty action: Categorization

Let's create a new column called Decada that we can use on the analysis. We will build based on the column Year.

In [39]:
DF_Clean.Year.value_counts().reset_index().sort_values("index").set_index('index').Year

index
0.0        56
5.0         1
77.0        1
500.0       1
1555.0      1
1638.0      1
1642.0      2
1721.0      1
1738.0      1
1742.0      1
1751.0      1
1764.0      1
1771.0      1
1776.0      1
1779.0      1
1780.0      1
1787.0      1
1788.0      1
1791.0      1
1797.0      1
1801.0      1
1803.0      2
1804.0      1
1805.0      1
1807.0      1
1810.0      1
1811.0      1
1816.0      1
1817.0      1
1819.0      1
         ... 
1989.0     52
1990.0     38
1991.0     35
1992.0     48
1993.0     39
1994.0     50
1995.0     65
1996.0     55
1997.0     52
1998.0     62
1999.0     55
2000.0     85
2001.0     83
2002.0     78
2003.0     85
2004.0     79
2005.0     87
2006.0     97
2007.0     99
2008.0    107
2009.0     97
2010.0     76
2011.0    104
2012.0    106
2013.0    106
2014.0    114
2015.0    131
2016.0    116
2017.0    116
2018.0     45
Name: Year, Length: 217, dtype: int64

In [40]:
DF_Clean["Decada"] = pd.cut(DF_Clean.Year,
                                bins=[1949,1959,1969,1979,1989,1999,2009,2019], 
                                labels=range(1950,2020,10))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
DF_Clean["Decada"].value_counts()

2010    914
2000    897
1990    499
1960    467
1980    381
1950    315
1970    271
Name: Decada, dtype: int64

# Last Clean Actions:

## Remove the columns we won't use

Now that we saw the date, we know better those columns that we can delete, because we won't use on the analysis.

In [42]:
DF_Clean = DF_Clean.drop(columns=['Case Number','Date','Area','Time'])


In [43]:
DF_Clean['Year'] = DF_Clean['Year'].fillna("Not Available")
DF_Clean['Type'] = DF_Clean['Type'].fillna("Not Available")
DF_Clean['Country'] = DF_Clean['Country'].fillna("Not Available")
DF_Clean['Activity'] = DF_Clean['Activity'].fillna("Not Available")
DF_Clean['Sex '] = DF_Clean['Sex '].fillna("Not Available")
DF_Clean['Age'] = DF_Clean['Age'].fillna("Not Available")
DF_Clean['Fatal (Y/N)'] = DF_Clean['Fatal (Y/N)'].fillna("Not Available")
DF_Clean['Species '] = DF_Clean['Species '].fillna("Not Available")

## Drop "Not Available" in the column Country

The information Country is essential for our analysis, so if we don't have this information we can disregard from the database.

In [44]:
DF_Clean = DF_Clean.drop(DF_Clean[DF_Clean['Country']=="Not Available"].index)

In [45]:
#DF_Clean

## Remove all "NaN" from the column Decada

If Decada is NaN it means that this information os either from before 1949 or because the data Year was not fulfill corrected. So, we can drop these data because is useless.

We can use the formula below straight on the database, because the only column that have "NaN" is Decada.

In [46]:
DF_Clean = DF_Clean.replace(-1, np.NaN).dropna()

## Check Again with we have duplicated values

Now that we already done cleaning the database we can see if we have any duplicated row and in case yes, we can delete it.

In [47]:
DF_Clean.drop_duplicates()

display(DF_Clean.head())
print(DF_Clean.shape)

,Year,Type,Country,Activity,Sex,Age,Fatal (Y/N),Species,Decada
1,2018,Unprovoked,USA,Treading water,F,Teen (1 - 19),N,Not Available,2010
2,2018,Not Available,USA,Surfing,M,Adult (30 - 59),N,Not Available,2010
3,2018,Unprovoked,AUSTRALIA,Surfing,M,Not Available,N,Not Available,2010
4,2018,Provoked,MEXICO,Diving,M,Not Available,N,Tiger Shark,2010
5,2018,Unprovoked,AUSTRALIA,Surfing,M,Not Available,N,Not Available,2010


(3732, 9)


## Data Base clean!!!

Started: 25.723 X 24

Finished: 3.732 X 9

## Export the new database to a CSV

In [49]:
DF_Clean.to_csv(r"/home/carolina/Desktop/IRONHACK/Project files/clean_df_attacks.csv", index = False)